In [80]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-product-reviews-dataset/marketing_sample_for_walmart_com-walmart_product_reviews__20200401_20200630__30k_data.csv
/kaggle/input/walmart-review-data/WalmartProducts.csv


# **Load Library**

In [81]:
from warnings import filterwarnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from textblob import Word, TextBlob
from sklearn.model_selection import train_test_split #cross_val_score, GridSearchCV, cross_validate, train_test_split, RandomizedSearchCV
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, auc
#from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import Word, TextBlob

import tensorflow as tf
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup,TFBertForSequenceClassification
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

# **Load Data**

In [82]:
os.listdir("../input/walmart-product-reviews-dataset/")
walmart2 = pd.read_csv("../input/walmart-product-reviews-dataset/marketing_sample_for_walmart_com-walmart_product_reviews__20200401_20200630__30k_data.csv")
walmart = pd.read_csv("../input/walmart-review-data/WalmartProducts.csv")
print(walmart.shape)

(30006, 19)


# **Filtering and single feature exploration**

In [83]:
walmart.head()

,index,Uniq Id,Crawl Timestamp,Pageurl,Website,Title,Rating,Review,Reviewer Name,Review Upvotes,Review Downvotes,Verified Purchaser,Recommended Purchase,Review Date,Five Star,Four Star,Three Star,Two Star,One Star
0,0,ee40ad5e6211d47a92fa1962938fad16,2020-03-17 03:37:35 +0000,https://www.walmart.com/reviews/product/15066091,walmart.com,"Sylvania 9"" Portable Dvd Player With Swivel Sc...",5.0,This product works great. It's exactly what I ...,Diane,0,0,No,No,12/4/2018,133.0,58.0,15.0,9.0,34.0
1,1,d10f87c07c206c72b0b5ddbf90541b38,2020-03-27 13:15:01 +0000,https://www.walmart.com/reviews/product/247042233,walmart.com,NaN,1.0,My carrier said this phone had been activated ...,Bren,0,0,No,No,12/21/2018,9.0,0.0,1.0,2.0,8.0
2,2,ef254e936e02a1b2bdaf89dd51b73403,2020-02-17 04:49:37 +0000,https://www.walmart.com/reviews/product/649827918,walmart.com,"VIZIO 65"" Class 4K UHD Quantum SmartCast Smart...",5.0,The picture on this TV is better than any TV I...,Hieronymesque,0,0,Yes,No,12/25/2019,180.0,39.0,5.0,8.0,13.0
3,3,183ab6b4ec58daae79c66d0151400b63,2020-02-17 03:11:54 +0000,https://www.walmart.com/reviews/product/665198047,walmart.com,TSV For Fitbit Charge 3 USB Charging Cradle Wa...,5.0,NaN,D,0,0,No,No,4/2/2019,2.0,3.0,0.0,0.0,0.0
4,4,03628759aaed53a0b9d137bca9991c5d,2020-02-07 01:17:14 +0000,https://www.walmart.com/reviews/product/125301514,walmart.com,AKG K92 Closed-Back Headphones - Stereo - Wire...,5.0,Very well-balanced between the highs and the l...,Dubbsta,52,1,No,No,12/9/2017,9.0,3.0,0.0,0.0,2.0


In [84]:
walmart.columns

Index(['index', 'Uniq Id', 'Crawl Timestamp', 'Pageurl', 'Website', 'Title',
       'Rating', 'Review', 'Reviewer Name', 'Review Upvotes',
       'Review Downvotes', 'Verified Purchaser', 'Recommended Purchase',
       'Review Date', 'Five Star', 'Four Star', 'Three Star', 'Two Star',
       'One Star'],
      dtype='object')

In [85]:
walmart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30006 entries, 0 to 30005
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 30006 non-null  int64  
 1   Uniq Id               30006 non-null  object 
 2   Crawl Timestamp       30006 non-null  object 
 3   Pageurl               30006 non-null  object 
 4   Website               30006 non-null  object 
 5   Title                 24412 non-null  object 
 6   Rating                30006 non-null  float64
 7   Review                25821 non-null  object 
 8   Reviewer Name         28358 non-null  object 
 9   Review Upvotes        30006 non-null  int64  
 10  Review Downvotes      30006 non-null  int64  
 11  Verified Purchaser    30006 non-null  object 
 12  Recommended Purchase  30006 non-null  object 
 13  Review Date           30006 non-null  object 
 14  Five Star             29910 non-null  float64
 15  Four Star          

In [86]:
walmart.isna().sum()

index                      0
Uniq Id                    0
Crawl Timestamp            0
Pageurl                    0
Website                    0
Title                   5594
Rating                     0
Review                  4185
Reviewer Name           1648
Review Upvotes             0
Review Downvotes           0
Verified Purchaser         0
Recommended Purchase       0
Review Date                0
Five Star                 96
Four Star                 96
Three Star                96
Two Star                  96
One Star                  96
dtype: int64

In [87]:
walmart = walmart.drop(columns=['Five Star','Four Star','Three Star','Two Star','One Star'])
walmart.shape

(30006, 14)

In [88]:
walmart = walmart[walmart['Review'].notna()]
walmart.shape

(25821, 14)

In [89]:
walmart=walmart.drop(columns=['Recommended Purchase'])
walmart.shape

(25821, 13)

# **Text Preprocessing**

In [90]:
# transform to lower case
walmart['Review'] = walmart['Review'].str.lower()

In [91]:
# remove punctuations
walmart['Review'] = walmart['Review'].str.replace('[^\w\s]','',regex=True)

In [92]:
#remove number in the review text
walmart['Review'] = walmart['Review'].str.replace('\d', '', regex=True)

In [93]:
#remove stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
walmart['Review'] = walmart['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
#remove rare words: remove words only appearing one time
words_count = pd.Series(' '.join(walmart['Review']).split()).value_counts()
rare_word = words_count[words_count<=1]
walmart['Review'] = walmart['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_word))

In [95]:
#Lemmatization:  reduces different inflected forms of words (like 'running' to 'run') to their base form.  
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [96]:
cmd = "unzip -d /usr/share/nltk_data/corpora/ /usr/share/nltk_data/corpora/wordnet.zip"
os.system(cmd)
os.system("ls /usr/share/nltk_data/corpora/wordnet*")

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
/usr/share/nltk_data/corpora/wordnet.zip
/usr/share/nltk_data/corpora/wordnet_ic.zip

/usr/share/nltk_data/corpora/wordnet:
LICENSE
README
adj.exc
adv.exc
citation.bib
cntlist.rev
data.adj
data.adv
data.noun
data.verb
index.adj
index.adv
index.noun
index.sense
index.verb
lexnames
noun.exc
verb.exc

/usr/share/nltk_data/corpora/wordnet_ic:
IC-compute.sh
IC-semcor.sh
README
ic-bnc-add1.dat
ic-bnc-resnik-add1.dat
ic-bnc-resnik.dat
ic-bnc.dat
ic-brown-add1.dat
ic-brown-resnik-add1.dat
ic-brown-resnik.dat
ic-brown.dat
ic-semcor-add1.dat
ic-semcor.dat
ic-semcorraw-add1.dat
ic-semcorraw-resnik-add1.dat
ic-semcorraw-resnik.dat
ic-semcorraw.dat
ic-shaks-add1.dat
ic-shaks-resnik.dat
ic-shaks-resnink-add1.dat
ic-shaks.dat
ic-treebank-add1.dat
ic-treebank-resnik-add1.dat
ic-treebank-resnik.dat
ic-treebank.dat
shaks12.txt
stoplist.txt
wn30compounds.txt


replace /usr/share/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


0

In [97]:
#I found that some verbs are not properly lemmatized
Word("running").lemmatize()

'running'

In [98]:
#Propoer tag should be added to make it works
Word("running").lemmatize('v')

'run'

In [99]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

# Lemmatize
sentence = "The striped bats are hanging on their feet for best"
lemmatize_with_postag(sentence)
#> 'The striped bat be hang on their foot for best'

'The striped bat be hang on their foot for best'

In [100]:
walmart['Review'] = walmart['Review'].apply(lambda x: lemmatize_with_postag(x))

# **Sentiment Analysis**

In [101]:
#transform rating to positive and negative 
walmart["Senti"]=walmart["Rating"].apply(lambda x: "pos" if x > 3 else "neg")

In [102]:
walmart.head()

,index,Uniq Id,Crawl Timestamp,Pageurl,Website,Title,Rating,Review,Reviewer Name,Review Upvotes,Review Downvotes,Verified Purchaser,Review Date,Senti
0,0,ee40ad5e6211d47a92fa1962938fad16,2020-03-17 03:37:35 +0000,https://www.walmart.com/reviews/product/15066091,walmart.com,"Sylvania 9"" Portable Dvd Player With Swivel Sc...",5.0,product work great exactly look,Diane,0,0,No,12/4/2018,pos
1,1,d10f87c07c206c72b0b5ddbf90541b38,2020-03-27 13:15:01 +0000,https://www.walmart.com/reviews/product/247042233,walmart.com,NaN,1.0,carrier say phone activate yet purchase new ph...,Bren,0,0,No,12/21/2018,neg
2,2,ef254e936e02a1b2bdaf89dd51b73403,2020-02-17 04:49:37 +0000,https://www.walmart.com/reviews/product/649827918,walmart.com,"VIZIO 65"" Class 4K UHD Quantum SmartCast Smart...",5.0,picture tv well tv ive ever own life extremely...,Hieronymesque,0,0,Yes,12/25/2019,pos
4,4,03628759aaed53a0b9d137bca9991c5d,2020-02-07 01:17:14 +0000,https://www.walmart.com/reviews/product/125301514,walmart.com,AKG K92 Closed-Back Headphones - Stereo - Wire...,5.0,wellbalanced high low bass treble good volume ...,Dubbsta,52,1,No,12/9/2017,pos
5,5,7102ff804ecde89329af14dcf21d04b1,2020-03-01 10:26:04 +0000,https://www.walmart.com/reviews/product/149475379,walmart.com,Panasonic HD WiFi Camera KX-HN0002B,1.0,camera every one experience issue ghz support ...,Dave in SoCal,5,1,No,8/8/2018,neg


# **Modeling Exploration**

In [103]:
# Test-Train
train_x, test_x, train_y, test_y = train_test_split(walmart["Review"],
                                                    walmart["Senti"],
                                                    random_state=428)

# **BERT Model**

In [104]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [105]:
x_train, x_val, y_train, y_val = train_test_split(train_x,
                                                    train_y,
                                                    test_size=0.25, 
                                                    stratify = train_y)

In [106]:
max_len= 256
X_train_encoded = tokenizer.batch_encode_plus(x_train.tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')
X_val_encoded = tokenizer.batch_encode_plus(x_val.tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')
X_test_encoded = tokenizer.batch_encode_plus(test_x.tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

In [107]:
senti_dict = {"pos": 1, "neg": 0}
Y_train_label = y_train.apply(lambda x: senti_dict[x])
Y_val_label = y_val.apply(lambda x: senti_dict[x])
Y_test_label = test_y.apply(lambda x: senti_dict[x])

In [108]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [109]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [110]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
    
print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


In [111]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [112]:
history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
    Y_train_label,
    validation_data=(
      [X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],Y_val_label),
    batch_size=32,
    epochs=3
)

Epoch 1/3
454/454 [==============================] - 521s 1s/step - loss: 0.3719 - accuracy: 0.8378 - val_loss: 0.2647 - val_accuracy: 0.8912
Epoch 2/3
454/454 [==============================] - 475s 1s/step - loss: 0.2326 - accuracy: 0.9065 - val_loss: 0.2481 - val_accuracy: 0.9040
Epoch 3/3
454/454 [==============================] - 475s 1s/step - loss: 0.1727 - accuracy: 0.9339 - val_loss: 0.3062 - val_accuracy: 0.8982


In [113]:
test_loss, test_accuracy = model.evaluate(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
    Y_test_label
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

202/202 [==============================] - 65s 320ms/step - loss: 0.3093 - accuracy: 0.8987
Test loss: 0.3092724084854126, Test accuracy: 0.8986988663673401


In [114]:
Y_test_pred = model.predict(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])
 

202/202 [==============================] - 67s 316ms/step


In [115]:
logits = Y_test_pred.logits
pred_labels = tf.argmax(logits, axis=1)
pred_labels = pred_labels.numpy()


In [116]:
print(classification_report(Y_test_label, pred_labels))

              precision    recall  f1-score   support

           0       0.81      0.70      0.75      1405
           1       0.92      0.95      0.94      5051

    accuracy                           0.90      6456
   macro avg       0.86      0.83      0.84      6456
weighted avg       0.90      0.90      0.90      6456



In [117]:
save_path = 'roberta_v3_small_pretrained_model_pytorch'
!mkdir {save_path}

In [118]:
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('roberta_v3_small_pretrained_model_pytorch/tokenizer_config.json',
 'roberta_v3_small_pretrained_model_pytorch/special_tokens_map.json',
 'roberta_v3_small_pretrained_model_pytorch/vocab.txt',
 'roberta_v3_small_pretrained_model_pytorch/added_tokens.json')

In [119]:
!zip -r roberta_small.zip {save_path}

  adding: roberta_v3_small_pretrained_model_pytorch/ (stored 0%)
  adding: roberta_v3_small_pretrained_model_pytorch/tf_model.h5 (deflated 8%)
  adding: roberta_v3_small_pretrained_model_pytorch/vocab.txt (deflated 53%)
  adding: roberta_v3_small_pretrained_model_pytorch/tokenizer_config.json (deflated 75%)
  adding: roberta_v3_small_pretrained_model_pytorch/config.json (deflated 47%)
  adding: roberta_v3_small_pretrained_model_pytorch/special_tokens_map.json (deflated 42%)


In [124]:
walmart2.head()

,Uniq Id,Crawl Timestamp,Pageurl,Website,Title,Rating,Review,Reviewer Name,Review Upvotes,Review Downvotes,Verified Purchaser,Recommended Purchase,Review Date,Five Star,Four Star,Three Star,Two Star,One Star
0,1800878dcfa5255c691f2a87ce8b2869,2020-05-16 17:58:33 +0000,https://www.walmart.com/reviews/product/36907838,walmart.com,NaN,1.0,One star for looking nice. That’s it. After tr...,PJ10,0,0,No,No,7/28/2018,23.0,5.0,1.0,3.0,6.0
1,ac06ca1b77310d539c9a74eed7a8a8e2,2020-04-17 01:05:43 +0000,https://www.walmart.com/reviews/product/708236785,walmart.com,NaN,4.0,Love this phone so far have had it almost a mo...,KelJo,9,3,No,No,11/14/2017,91.0,23.0,13.0,9.0,31.0
2,9c4b10fff4e5e2cc7e2e1bbed76e2220,2020-06-08 10:35:05 +0000,https://www.walmart.com/reviews/product/831078728,walmart.com,NaN,4.0,This TV is absolutely fantastic. This is my th...,TheBigChalupa11,0,0,Yes,No,5/20/2020,32.0,8.0,1.0,0.0,0.0
3,b5a5dc8de9fc0468aab961f097b56091,2020-04-13 03:57:29 +0000,https://www.walmart.com/reviews/product/40712755,walmart.com,NaN,5.0,"Refurb, good shape, good price, does what I wa...",Micalic,0,0,No,No,7/9/2019,418.0,62.0,17.0,1.0,13.0
4,430433b52b882e540c16363d448e012b,2020-05-19 03:52:42 +0000,https://www.walmart.com/reviews/product/46597970,walmart.com,NaN,5.0,Very nice tablet! Looks brand new. Fired right...,jcan74,0,0,No,No,2/11/2020,213.0,57.0,25.0,24.0,87.0


In [126]:
walmart2 = walmart2.drop(columns=['Five Star','Four Star','Three Star','Two Star','One Star'])
walmart2.shape

(29997, 13)

In [131]:
#overlap between walmart2 and walmart
overlap = set(walmart2["Uniq Id"]).intersection(set(walmart["Uniq Id"]))
len(overlap)

1009

In [135]:
#remove overlap
walmart2 = walmart2[~walmart2["Uniq Id"].isin(overlap)]

In [136]:
walmart2 = walmart2[walmart2['Review'].notna()]
walmart2.shape

(23609, 13)

In [137]:
walmart2=walmart2.drop(columns=['Recommended Purchase'])
walmart2.shape

(23609, 12)

In [138]:
# transform to lower case
walmart2['Review'] = walmart2['Review'].str.lower()


In [139]:
# remove punctuations
walmart2['Review'] = walmart2['Review'].str.replace('[^\w\s]','',regex=True)

In [140]:
#remove number in the review text
walmart2['Review'] = walmart2['Review'].str.replace('\d', '', regex=True)

In [141]:
#remove stopwords
walmart2['Review'] = walmart2['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

In [142]:
#remove rare words
words_count = pd.Series(' '.join(walmart2['Review']).split()).value_counts()
rare_word = words_count[words_count<=1]
walmart2['Review'] = walmart2['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_word))

In [143]:
walmart2['Review'] = walmart2['Review'].apply(lambda x: lemmatize_with_postag(x))

In [144]:
walmart2["Senti"]=walmart2["Rating"].apply(lambda x: "pos" if x > 3 else "neg")
walmart2.head()

,Uniq Id,Crawl Timestamp,Pageurl,Website,Title,Rating,Review,Reviewer Name,Review Upvotes,Review Downvotes,Verified Purchaser,Review Date,Senti
0,1800878dcfa5255c691f2a87ce8b2869,2020-05-16 17:58:33 +0000,https://www.walmart.com/reviews/product/36907838,walmart.com,NaN,1.0,one star look nice thats try try par iphone fi...,PJ10,0,0,No,7/28/2018,neg
1,ac06ca1b77310d539c9a74eed7a8a8e2,2020-04-17 01:05:43 +0000,https://www.walmart.com/reviews/product/708236785,walmart.com,NaN,4.0,love phone far almost month bad problem reason...,KelJo,9,3,No,11/14/2017,pos
2,9c4b10fff4e5e2cc7e2e1bbed76e2220,2020-06-08 10:35:05 +0000,https://www.walmart.com/reviews/product/831078728,walmart.com,NaN,4.0,tv absolutely fantastic third k tv blow away e...,TheBigChalupa11,0,0,Yes,5/20/2020,pos
3,b5a5dc8de9fc0468aab961f097b56091,2020-04-13 03:57:29 +0000,https://www.walmart.com/reviews/product/40712755,walmart.com,NaN,5.0,refurb good shape good price want,Micalic,0,0,No,7/9/2019,pos
4,430433b52b882e540c16363d448e012b,2020-05-19 03:52:42 +0000,https://www.walmart.com/reviews/product/46597970,walmart.com,NaN,5.0,nice tablet look brand new fire right problem ...,jcan74,0,0,No,2/11/2020,pos


In [156]:
walmart2_x_encoded = tokenizer.batch_encode_plus(walmart2["Review"].tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

In [146]:
walmart2_Y_label = walmart2["Senti"].apply(lambda x: senti_dict[x])

In [152]:
model = TFBertForSequenceClassification.from_pretrained(save_path, return_dict=True)
tokenizer = BertTokenizer.from_pretrained(save_path)

Some layers from the model checkpoint at roberta_v3_small_pretrained_model_pytorch were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at roberta_v3_small_pretrained_model_pytorch.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [154]:
model.summary

<bound method Model.summary of <transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification object at 0x7fc84218d390>>

In [157]:
walmart2_Y_pred = model.predict(
    [walmart2_x_encoded['input_ids'], walmart2_x_encoded['token_type_ids'], walmart2_x_encoded['attention_mask']])

738/738 [==============================] - 235s 315ms/step


In [158]:
logits = walmart2_Y_pred.logits
pred_labels = tf.argmax(logits, axis=1)
pred_labels = pred_labels.numpy()

In [159]:
print(classification_report(walmart2_Y_label, pred_labels))

              precision    recall  f1-score   support

           0       0.83      0.69      0.75      5360
           1       0.91      0.96      0.94     18249

    accuracy                           0.90     23609
   macro avg       0.87      0.82      0.84     23609
weighted avg       0.89      0.90      0.89     23609

